<a href="https://colab.research.google.com/github/Lujein-alsheikh/Hackathon_wns_trainge/blob/main/hackathon_code_notebook_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import PIL
from PIL import Image

# import seaborn as sns

In [ ]:
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras.callbacks import EarlyStopping

In [ ]:
from keras.layers import MaxPool2D, Conv2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from keras.models import Sequential
from keras.optimizers import Adam

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
original_training_df = pd.read_csv("drive/MyDrive/train_hackathon/train.csv")
original_test_df= pd.read_csv("drive/MyDrive/test_hackathon/test.csv")

In [ ]:
original_training_df.head()

,image_id,filename,label
0,1,1.jpg,0
1,2,2.jpg,0
2,3,3.jpg,0
3,4,4.jpg,0
4,5,5.jpg,0


In [ ]:
original_training_df.shape

(8079, 3)

In [ ]:
train_df, val_df = train_test_split(original_training_df, test_size=0.2, random_state=42, stratify= original_training_df["label"])

In [ ]:
# I have to cast the labels because model.compile complained about int64. it expects float32 (for some reason)
train_df["label"] = train_df["label"].astype('float32')
val_df["label"] = val_df["label"].astype('float32')

In [ ]:
train_df[train_df["label"]==1].head()

,image_id,filename,label
6816,6817,6817.jpg,1.0
2003,2004,2004.jpg,1.0
5841,5842,5842.jpg,1.0
1643,1644,1644.jpg,1.0
5970,5971,5971.jpg,1.0


In [ ]:
val_df[val_df["filename"]=="18.jpg"]

,image_id,filename,label
17,18,18.jpg,1.0


In [ ]:
train_df["label"].value_counts()

0.0    6091
1.0     372
Name: label, dtype: int64

In [ ]:
val_df["label"].value_counts()

0.0    1523
1.0      93
Name: label, dtype: int64

In [ ]:
train_df.shape, val_df.shape

((6463, 3), (1616, 3))

**loading the images into a tensorflow dataframe**

In [ ]:
file_paths_train = train_df["filename"].values
labels_train = train_df["label"].values
ds_train = tf.data.Dataset.from_tensor_slices((file_paths_train, labels_train))

In [ ]:
ds_train.reduce(tf.constant(0), lambda x, _: x + 1)

<tf.Tensor: shape=(), dtype=int32, numpy=6463>

In [ ]:
type(ds_train)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [ ]:
file_paths_val = val_df["filename"].values
labels_val = val_df["label"].values
ds_val = tf.data.Dataset.from_tensor_slices((file_paths_val, labels_val))

In [ ]:
type(ds_val), ds_val.reduce(tf.constant(0), lambda x, _: x + 1)

(tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset,
 <tf.Tensor: shape=(), dtype=int32, numpy=1616>)

In [ ]:
height = 600
width = 800
channels=3
batch_size_train =128
batch_size_val=32

In [ ]:
dir = "drive/MyDrive/train_hackathon/resized_images/"

def read_image(image_file, label):
  image= tf.io.read_file(dir + image_file) # returns an encoded image
  image = tf.image.decode_image(image, channels=3, dtype= tf.float32) # returns a tensorflow.python.framework.ops.EagerTensor
  return image, label

In [ ]:
output_img = read_image("1.jpg",0)[0]

In [ ]:
type(output_img)

tensorflow.python.framework.ops.EagerTensor

In [ ]:
output_img.get_shape(), output_img.shape

(TensorShape([600, 800, 3]), TensorShape([600, 800, 3]))

In [ ]:
# resize_layer = tf.keras.layers.Resizing(height = height, width= width)
rescale_layer = tf.keras.layers.Rescaling(scale= 1./255)

the resize_layer had a problem with the dynamic shapes of the images (that each image has a different shape). tf.image.resize as well. I resized them using pillow (PIL) and saved them on drive.

In [ ]:
def trivial_preprocessing(image,label): # rescaling and resizing only
      image = rescale_layer(image)
      # image = resize_layer(image)
      # image = tf.image.resize(image, [height, width])
      return image,label

In [ ]:
ds_train= ds_train.shuffle(buffer_size= batch_size_train*10).map(read_image).map(trivial_preprocessing).batch(batch_size_train)
ds_val= ds_val.shuffle(buffer_size= batch_size_val*10).map(read_image).map(trivial_preprocessing).batch(batch_size_val)

**To understand .shuffle and .batch** watch this https://www.youtube.com/watch?v=c7G5W4Wv72Q

In [ ]:
type(ds_train)

tensorflow.python.data.ops.batch_op._BatchDataset

**Baseline model without data augmentation**

In [ ]:
baseline_model = Sequential()
baseline_model.add(InputLayer(input_shape=(height, width, 3)))
baseline_model.add(Conv2D(5, (5, 5), activation='relu', strides=(2,2), padding='valid'))
baseline_model.add(MaxPool2D(pool_size=(5, 5), padding='same'))
baseline_model.add(Flatten())
baseline_model.add(Dense(units=128, activation='sigmoid'))
baseline_model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 398, 5)       380       
                                                                 
 max_pooling2d (MaxPooling2  (None, 60, 80, 5)         0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 24000)             0         
                                                                 
 dense (Dense)               (None, 128)               3072128   
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3072637 (11.72 MB)
Trainable params: 3072637 (11.72 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
metric = tf.keras.metrics.F1Score(average= "macro", threshold= 0.5, name='macro_f1_score')
adam = Adam(learning_rate =1e-5)
baseline_model.compile(optimizer = adam, loss='binary_crossentropy', metrics=[metric])

In [ ]:
train_df.dtypes

image_id      int64
filename     object
label       float32
dtype: object

In [ ]:
num_epochs = 50
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3, mode='min')
baseline_model_history=  baseline_model.fit(ds_train, epochs=num_epochs, validation_data=ds_val, callbacks=[early_stopping])

Epoch 1/50
51/51 [==============================] - 1731s 33s/step - loss: 0.5145 - macro_f1_score: 0.0000e+00 - val_loss: 0.4972 - val_macro_f1_score: 0.0000e+00
Epoch 2/50
51/51 [==============================] - 549s 11s/step - loss: 0.4744 - macro_f1_score: 0.0000e+00 - val_loss: 0.4457 - val_macro_f1_score: 0.0000e+00
Epoch 3/50
51/51 [==============================] - 538s 11s/step - loss: 0.4140 - macro_f1_score: 0.0000e+00 - val_loss: 0.3782 - val_macro_f1_score: 0.0000e+00
Epoch 4/50
51/51 [==============================] - 518s 10s/step - loss: 0.3463 - macro_f1_score: 0.0000e+00 - val_loss: 0.3140 - val_macro_f1_score: 0.0000e+00
Epoch 5/50
51/51 [==============================] - 533s 10s/step - loss: 0.2912 - macro_f1_score: 0.0000e+00 - val_loss: 0.2698 - val_macro_f1_score: 0.0000e+00
Epoch 6/50
51/51 [==============================] - 502s 10s/step - loss: 0.2565 - macro_f1_score: 0.0000e+00 - val_loss: 0.2446 - val_macro_f1_score: 0.0000e+00
Epoch 7/50
51/51 [=========

KeyboardInterrupt: 

**Note:** The positive class is the minority class; class of fraud images. Usually in binary classification problems, the 1 class is considered to be the positive class.

**problem: notice the 0 values for the macro f1 score. It doesn't make sense that macro f1 score = 0. What is shown is actually the f1 score for the minority class only despite specifying "macro" when defining the f1 metric. Let us check this by checking the classification report after training the baseline model for one epoch only.**

In [ ]:
baseline_model.compile(optimizer=adam, loss='binary_crossentropy', metrics=[metric])
baseline_model.fit(ds_train, epochs=1)
val_results = baseline_model.evaluate(ds_val)
print("On the validation set: ", "val loss = ", val_results[0], "Validation Macro F1 Score = ", val_results[1])

51/51 [==============================] - 50s 967ms/step - loss: 0.2209 - macro_f1_score: 0.0000e+00
On the validation set:  val loss =  0.2209019809961319 Validation Macro F1 Score =  0.0


In [ ]:
val_pred = baseline_model.predict(ds_val) # predictions are floats (probabilities of belonging to the positive class)

val_true_labels = []
for _, label in ds_val.unbatch().as_numpy_iterator():
    val_true_labels.append(label)

# val_true_labels is a list of floats.

51/51 [==============================] - 52s 1s/step


'\nprint("Classification Report:")\nprint(classification_report(val_true_labels, val_pred.argmax(axis=1)))\n'

In [ ]:
classification_threshold = 0.5
val_pred_binary = (val_pred > classification_threshold).astype(int)

In [ ]:
val_true_labels = [int(label) for label in val_true_labels]

In [ ]:
print(classification_report(val_true_labels, val_pred_binary))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1523
           1       0.00      0.00      0.00        93

    accuracy                           0.94      1616
   macro avg       0.47      0.50      0.49      1616
weighted avg       0.89      0.94      0.91      1616



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
f1_score(val_true_labels, val_pred_binary, average='macro')

0.4851863650844218

**Workaround: Define a custom callback**

In [ ]:
class MacroF1ScoreCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, ds_train, ds_val):
        super(MacroF1ScoreCallback, self).__init__()
        self.model = model
        self.ds_train = ds_train
        self.ds_val = ds_val
        self.macro_f1_train = []
        self.macro_f1_val = []

    def calculate_macro_f1(self, dataset):
        y_true = tf.concat([y for x, y in dataset], axis=0)
        y_pred_probs = self.model.predict(dataset)
        y_pred = (y_pred_probs > 0.5).astype(int)
        macro_f1 = f1_score(y_true.numpy(), y_pred, average='macro')
        return macro_f1

    def on_epoch_end(self, epoch, logs=None):
        macro_f1_train = self.calculate_macro_f1(self.ds_train)
        self.macro_f1_train.append(macro_f1_train)
        macro_f1_val = self.calculate_macro_f1(self.ds_val)
        self.macro_f1_val.append(macro_f1_val)
        print(f'Epoch {epoch + 1} - Macro F1 Score (Train): {macro_f1_train}')
        print(f'Epoch {epoch + 1} - Macro F1 Score (Validation): {macro_f1_val}')

macro_f1_callback = MacroF1ScoreCallback(model=baseline_model, ds_train=ds_train, ds_val=ds_val)

In [ ]:
f1_metric = tf.keras.metrics.F1Score(average=None, threshold= 0.5, name='f1_score')
baseline_model.compile(optimizer=adam, loss='binary_crossentropy', metrics=[f1_metric])

# Fit the model with the custom callback
baseline_model.fit(ds_train, epochs=1, validation_data=ds_val, callbacks=[macro_f1_callback])

51/51 [==============================] - 50s 965ms/step
Epoch 1 - Macro F1 Score (Train): 0.4851840050979767
Epoch 1 - Macro F1 Score (Validation): 0.4851863650844218
51/51 [==============================] - 913s 18s/step - loss: 0.2209 - f1_score: 0.0000e+00 - val_loss: 0.2208 - val_f1_score: 0.0000e+00


**Image Augmentation**

In [ ]:
seed = tf.constant([random.randint(0, 9999), 0], dtype=tf.int32)

In [ ]:
def add_noise(img, noise_factor=0.2):
    noise = tf.random.normal(shape=tf.shape(img), mean=0.0, stddev=noise_factor, dtype=tf.float32)
    noisy_image = img + noise
    noisy_image = tf.clip_by_value(noisy_image, 0.0, 1.0)
    return noisy_image

def flip_left_right(image):
  return tf.image.flip_left_right(image)

def random_hue(image):
  return tf.image.stateless_random_hue(image, max_delta= 0.469, seed= seed)

def random_saturation(image):
  return tf.image.stateless_random_saturation(image, lower=0.469, upper=90, seed = seed)

#def random_quality(image):
 # return tf.image.stateless_random_jpeg_quality(image, min_jpeg_quality=60, max_jpeg_quality=90, seed= seed)

**Note that the seed is obligatory in tf.image functions whereas it is an optional argument in the tf.keras layers.** <br/>
**I don't understand why it should have two elements.** <br/>
A tf.image function with the same seed always outputs the same augmented image.

In [ ]:
random_brightness = tf.keras.layers.RandomBrightness(factor= 0.8, value_range=[0.0, 1.0], seed=None)
random_contrast = tf.keras.layers.RandomContrast(factor=0.5, seed=None)
random_translation = tf.keras.layers.RandomTranslation(height_factor= 0.5,width_factor=0.5,fill_mode='reflect',interpolation='bilinear',seed=None)
random_rotation = tf.keras.layers.RandomRotation(factor=0.3, fill_mode='reflect',interpolation='bilinear',seed=None)
random_zoom = tf.keras.layers.RandomZoom(height_factor= (-0.3, -0.2), width_factor=None,fill_mode='reflect',interpolation='bilinear',seed=None)

In [ ]:
augmentation_functions= [add_noise, flip_left_right, random_hue, random_saturation, random_brightness, random_contrast,
                        random_translation,  random_rotation, random_zoom]

In [ ]:
len(augmentation_functions)

9

In [ ]:
def augment(image, label):

    augmented_images = [image]  # The original image is always included

    if label == 1:
        for fun in augmentation_functions:
              augmented_images.append(fun(image))

    for aug_img in augmented_images:
        yield aug_img, label

    # return augmented_images, tf.repeat(label, len(augmented_images))
    # returns a tuple with two elements. The first is a list of 10 tensor images and the second is a tensor with the 10 labels.

In [ ]:
# Convert the generator output to a TensorFlow dataset
def augment_wrapper(image, label):

    augmented_dataset = tf.data.Dataset.from_generator(
        lambda: augment(image, label),
        output_signature=(
            tf.TensorSpec(shape=( len(augmentation_functions) + 1, height, width, channels), dtype=tf.float32),  # image is a tensor
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )
    return augmented_dataset

In [ ]:
def augment_once(image,label):
  augmented_images=[image]
  if label ==1:
    aug_img = random_brightness(image)
    augmented_images.append(aug_img)
  #return augmented_images, tf.repeat(label,len(augmented_images))
  for img in augmented_images:
    yield img,label

**Why yield instead of return?** <br/>
We could've used ***return augmented_images, tf.repeat(label, len(augmented_images))*** which returns a tuple with two elements. The first is a list of 10 tensor images and the second is a tensor with the 10 labels. So far so good. The problem is when we apply the ds_train.map(augment) which will add to ds_train this tuple representing 10 images together which will be considered an element of ds_train. The usual element of ds_train, however, is a tuple with one image and one label only. This will make the tf.keras.layers confuse the shapes of the input images and lead to ValueError that the tensor shape is unknown. <br/>
The yield keyword returns a generator which can't be added to ds_train either. To fix this we use **.flat_map(lambda x, y: tf.data.Dataset.from_generator(lambda: augment(x, y), output_signature=(tf.TensorSpec(shape=(None, None, None), dtype=tf.float32), tf.TensorSpec(shape=(), dtype=tf.int32))))**

In [ ]:
dir = "drive/MyDrive/train_hackathon/resized_images/6817.jpg"
image = tf.io.read_file(dir)
image = tf.image.decode_image(image, channels=3, dtype= tf.float32)

In [ ]:
train_df[train_df["filename"]== "6817.jpg"]

,image_id,filename,label
6816,6817,6817.jpg,1.0


In [ ]:
res = augment(image,1)

In [ ]:
res

<generator object augment at 0x7993cbbc35a0>

**Thinking moment: We have to augment first and then shuffle here**

**Reminder: We augment training data only!**

In [ ]:
# ds_train= ds_train.shuffle(buffer_size= batch_size_train*10).map(read_image).map(trivial_preprocessing).batch(batch_size_train)
ds_train_aug = ds_train.map(read_image).map(trivial_preprocessing).flat_map(lambda x, y: tf.data.Dataset.from_generator(lambda: augment(x, y), output_signature=(tf.TensorSpec(shape=(height, width, channels), dtype=tf.float32), tf.TensorSpec(shape=(), dtype=tf.int32))))

#ds_train_aug = ds_train_aug.shuffle(buffer_size= 6500).batch(batch_size_train)
ds_val= ds_val.shuffle(buffer_size= batch_size_val*10).map(read_image).map(trivial_preprocessing).batch(batch_size_val)

In [ ]:
tf.data.experimental.cardinality(ds_train_aug).numpy()

-2

In [ ]:
ds_train_aug=ds_train.map(read_image).map(trivial_preprocessing)

In [ ]:
tf.data.experimental.cardinality(ds_train_aug).numpy()

6463

In [ ]:
ds_train_aug = ds_train_aug.flat_map(augment_wrapper)
print(tf.data.experimental.cardinality(ds_train_aug).numpy())

-2


In [ ]:
count = 0
for _ in ds_train_aug:
    count += 1

print(count)

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: <tf.Tensor 'args_1:0' shape=() dtype=float32> is out of scope and cannot be used here. Use return values, explicit Python locals or TensorFlow collections to access it.
Please see https://www.tensorflow.org/guide/function#all_outputs_of_a_tffunction_must_be_return_values for more information.

<tf.Tensor 'args_1:0' shape=() dtype=float32> was defined here:
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start
    File "/usr/local/lib/python3.10/dist-packages/torn

How to save and load a NN model?
https://machinelearningmastery.com/save-load-keras-deep-learning-models/ <br/>
The model after loading might give different results: https://machinelearningmastery.com/different-results-each-time-in-machine-learning/